## Process and Run Experiment on the Infrared Dataset

This notebook contains all the code needed to train PeopleNet using TAO on an infrared dataset.

This notebook requires the TAO Launcher, Docker and NGC to be setup

The github readme has links to setup the prerequisites

This notebook requires the FLIR dataset split archives to be downloaded and placed in ../../datasets/infrared

It can be found here https://www.flir.com/oem/adas/adas-dataset-form/


This notebook takes the following steps

1) Combine and unzip the FLIR dataset  
2) Convert the dataset labels into kitti labels  
3) Move images and labels into a kitti directory structure  
4) Map local directories the the TAO launcher
5) Use TAO Offline Augmentation to resize the images  
6) Split the dataset into a test set and 20%, 40%, 60% and 80% train subsets  
7) Generate TF records for test set and all training sets  
8) Download pretrained PeopleNet weights  
9) Train models with and without PeopleNet weights  
10) Graph Results

In [ ]:
!python3 -m pip install matplotlib

In [ ]:
import os
#paths relative to local repository
repo_home = os.path.join(os.getcwd(), "../../")
model_home = os.path.join(repo_home, "workspace/models")
dataset_home = os.path.join(repo_home, "datasets/infrared")
exp_home = os.path.join(repo_home, "workspace/peoplenet_ir")

#paths for inside the container
dataset_home_cont = "/datasets/infrared/"
exp_home_cont = "/tlt_exp/peoplenet_ir/"

### Combine and unzip the data

In [ ]:
%cd $dataset_home

In [ ]:
#combine the zip file together and unpack
!cat FLIR_ADAS_1_3.zip* > FLIR_combined.zip
!unzip FLIR_combined.zip

### Convert to kitti labels

In [ ]:
#Read the json data that contains truth data
import json

js_file_path = os.path.join(dataset_home, "FLIR_ADAS_1_3/train/thermal_annotations.json")
js_file = open(js_file_path, "r")
label_data = json.load(js_file)
js_file.close()

In [ ]:
print(label_data.keys())

In [ ]:
#Convert the json data into kitti labels
import os
ir_kitti = os.path.join(dataset_home, "ir_kitti")
label_out = os.path.join(ir_kitti, "labels")
os.makedirs(label_out, exist_ok=True)
for label in label_data["annotations"]:
    label_name = "FLIR_" + str(label["image_id"] + 1).zfill(5) + ".txt"
    cat_id = label["category_id"]
    with open(os.path.join(label_out, label_name), "a+") as file:
        cat = label_data["categories"][cat_id-1]["name"]
        xmin = label["bbox"][0]
        ymin = label["bbox"][1]
        xmax = xmin + label["bbox"][2]
        ymax = ymin + label["bbox"][3]
        file.write(f"{cat} 0 0 0 {xmin} {ymin} {xmax} {ymax} 0 0 0 0 0 0 0\n")     

### Move images and labels into a kitti directory structure

In [ ]:
#copy images that match with a label into the kitti image folder
import shutil
image_out = os.path.join(ir_kitti, "images")
os.makedirs(image_out, exist_ok=True)

for label in os.listdir(label_out):
    image_name1 = label.split(".")[0] + ".jpeg"
    image_name2 = label.split(".")[0] + ".jpg"
    shutil.copy(os.path.join(dataset_home,"FLIR_ADAS_1_3/train/thermal_8_bit", image_name1), os.path.join(image_out, image_name2))

## Map local directories the the TAO launcher

In [ ]:
# Mapping up the local directories to the TAO docker.
import json
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.path.join(repo_home, "datasets"),
            "destination": "/datasets"
        },
        # Mapping the specs directory.
        {
            "source": os.path.join(repo_home, "workspace"),
            "destination": "/tlt_exp"
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

### Use TAO Offline Augmentation to resize the images

In [ ]:
#Generate the augmentation spec file
img_x = 960
img_y = 544
ext = ".jpg"
aug_spec = f"""
    # Setting up dataset config.
    dataset_config{{
      image_path: "images"
      label_path: "labels"
    }}
    output_image_width: {img_x}
    output_image_height: {img_y}
    output_image_channel: 3
    image_extension: "{ext}"
 """
aug_spec_path = os.path.join(ir_kitti, "aug_spec.txt")
with open(aug_spec_path, "w+") as file:
    file.write(aug_spec)

In [ ]:
#Run augment command
ir_kitti_cont = os.path.join(dataset_home_cont, "ir_kitti")
aug_spec_path_cont = os.path.join(ir_kitti_cont, "aug_spec.txt")
resized_output_cont = os.path.join(dataset_home_cont,"ir_resized")
resized_output = os.path.join(dataset_home,"ir_resized")

!tao augment -d $ir_kitti_cont -a $aug_spec_path_cont -o $resized_output_cont

### Split the dataset into a test set and 20%, 40%, 60% and 80% train subsets

In [ ]:
def create_subset(original, name_list, output_folder):
    
    #determine image ext
    ext = os.path.splitext(os.listdir(os.path.join(original, "images"))[0])[1]
    
    
    image_out = os.path.join(output_folder, "images")
    label_out = os.path.join(output_folder, "labels")
    
    os.makedirs(image_out, exist_ok=True)
    os.makedirs(label_out, exist_ok=True)
    
    with open(name_list, "r") as ls:
        for line in ls:
            line = line.strip()
            shutil.copy(os.path.join(original,"images", line + ext), os.path.join(image_out, line + ext))
            shutil.copy(os.path.join(original, "labels", line + ".txt"), os.path.join(label_out, line + ".txt"))	
            

In [ ]:
subset_lists = ["test_set.txt", "train_20.txt", "train_40.txt", "train_60.txt", "train_80.txt"]
for list_file in subset_lists:
    output_name = list_file.split(".")[0]
    output_path = os.path.join(dataset_home, output_name)
    input_list_path = os.path.join(exp_home, list_file)
    create_subset(resized_output, input_list_path, output_path)

### Generate TF records for test set and all training sets 

In [ ]:
 def gen_tf_spec(dataset_path):

    spec_str = f"""
    kitti_config {{
      root_directory_path: "{dataset_path}"
      image_dir_name: "images"
      label_dir_name: "labels"
      image_extension: ".jpg"
      partition_mode: "random"
      num_partitions: 2
      val_split: 20
      num_shards: 10
    }}
    """
    return spec_str

In [ ]:
#Create the spec and generate tf records for all sets
datasets = ["test_set", "train_20", "train_40", "train_60", "train_80"]
for path in datasets:
    dataset_path = os.path.join(dataset_home, path)
    dataset_path_cont = os.path.join(dataset_home_cont, path)
    
    record_path = os.path.join(dataset_path, "tfrecord_spec.txt")
    record_path_cont = os.path.join(dataset_path_cont, "tfrecord_spec.txt")
    
    record_output = os.path.join(dataset_path_cont, "tfrecords/")
    
    with open(record_path, "w+") as spec:
        spec.write(gen_tf_spec(dataset_path_cont))
    !tao detectnet_v2 dataset_convert -d $record_path_cont -o $record_output

### Download pretrained PeopleNet weights

In [ ]:
%cd $model_home
!ngc registry model download-version "nvidia/tlt_peoplenet:unpruned_v2.1"

### Train models with and without PeopleNet weights

In [ ]:
!tao detectnet_v2 train -e /tlt_exp/peoplenet_ir/experiments/peoplenet_20/training_spec.txt -r /tlt_exp/peoplenet_ir/experiments/peoplenet_20 -n "final_model" -k "tlt_encode"

In [ ]:
!tao detectnet_v2 train -e /tlt_exp/peoplenet_ir/experiments/peoplenet_40/training_spec.txt -r /tlt_exp/peoplenet_ir/experiments/peoplenet_40 -n "final_model" -k "tlt_encode"

In [ ]:
!tao detectnet_v2 train -e /tlt_exp/peoplenet_ir/experiments/peoplenet_60/training_spec.txt -r /tlt_exp/peoplenet_ir/experiments/peoplenet_60 -n "final_model" -k "tlt_encode"

In [ ]:
!tao detectnet_v2 train -e /tlt_exp/peoplenet_ir/experiments/peoplenet_80/training_spec.txt -r /tlt_exp/peoplenet_ir/experiments/peoplenet_80 -n "final_model" -k "tlt_encode"

Train without peoplenet (random starting weights)

In [ ]:
!tao detectnet_v2 train -e /tlt_exp/peoplenet_ir/experiments/random_20/training_spec.txt -r /tlt_exp/peoplenet_ir/experiments/random_20 -n "final_model" -k "tlt_encode"

In [ ]:
!tao detectnet_v2 train -e /tlt_exp/peoplenet_ir/experiments/random_40/training_spec.txt -r /tlt_exp/peoplenet_ir/experiments/random_40 -n "final_model" -k "tlt_encode"

In [ ]:
!tao detectnet_v2 train -e /tlt_exp/peoplenet_ir/experiments/random_60/training_spec.txt -r /tlt_exp/peoplenet_ir/experiments/random_60 -n "final_model" -k "tlt_encode"

In [ ]:
!tao detectnet_v2 train -e /tlt_exp/peoplenet_ir/experiments/random_80/training_spec.txt -r /tlt_exp/peoplenet_ir/experiments/random_80 -n "final_model" -k "tlt_encode"

### Graph Results

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def get_map_data(filepath):
    x_vals_map = []
    y_vals_map = []
    with open(filepath, "r") as f:
        epoch = 0
        for line in f:
            data = eval(line)
            if "cur_epoch" in data.keys():
                epoch = data["cur_epoch"]

            elif "mean average precision" in data.keys():
                mAP = data["mean average precision"]
                y_vals_map.append(mAP)
                x_vals_map.append(epoch)

    return x_vals_map, y_vals_map

PeopleNet mAP

In [ ]:
f1 = os.path.join(exp_home, "peoplenet_20/status.json")
f2 = os.path.join(exp_home, "peoplenet_40/status.json")
f3 = os.path.join(exp_home, "peoplenet_60/status.json")
f4 = os.path.join(exp_home, "peoplenet_80/status.json")

files = [f4,f3,f2,f1] #modify for trainings that are complete
plt.figure(figsize=(10,5))
plt.title('PeopleNet on IR Data \n mAP Over Epoch')
plt.xlabel("Epoch")
plt.ylabel("mAP %")
plt.ylim([0,100])
plt.yticks(range(0,101,10))
plt.tick_params(right=True, labelright=True)

for f in files:
    x,y = get_map_data(f)
    print(f + "\n max mAP: " + str(max(y)) + " \n")
    plt.plot(x,y)

leg = ["x4", "x3", "x2", "x1"]
plt.legend(leg, title="Dataset Size \nx1 = 1,572")
plt.show()

Without PeopleNet mAP

In [ ]:
f1 = os.path.join(exp_home, "random_20/status.json")
f2 = os.path.join(exp_home, "random_40/status.json")
f3 = os.path.join(exp_home, "random_60/status.json")
f4 = os.path.join(exp_home, "random_80/status.json")

files = [f4,f3,f2,f1] #modify for trainings that are complete
plt.figure(figsize=(10,5))
plt.title('Without PeopleNet on IR Data \n mAP Over Epoch')
plt.xlabel("Epoch")
plt.ylabel("mAP %")
plt.ylim([0,100])
plt.yticks(range(0,101,10))
plt.tick_params(right=True, labelright=True)

for f in files:
    x,y = get_map_data(f)
    print(f + "\n max mAP: " + str(max(y)) + " \n")
    plt.plot(x,y)

leg = ["x4", "x3", "x2", "x1"]
plt.legend(leg, title="Dataset Size \nx1 = 1,572")
plt.show()